In [ ]:
# Entendendo a geração de próximos tokens, diferenças devices, e lembrando o que foi dito

In [ ]:
# Verifica se uma GPU está disponível no sistema (caso não haja, o comando não funcionará)
# !nvidia-smi

# Importa as bibliotecas necessárias: time, numpy, torch (PyTorch), e transformers (para os modelos de linguagem)
import time
import numpy as np
import torch
import transformers

# Verifica se há uma GPU disponível; retorna True se houver
print(torch.cuda.is_available())

# Define o dispositivo de execução como 'cuda' se houver GPU disponível; caso contrário, 'cpu'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(device)

False
cpu


In [ ]:
# Cria o pipeline de geração de texto usando o modelo TinyLlama e o dispositivo selecionado
tiny_llama = transformers.pipeline(
    task='text-generation',
    model='TinyLlama/TinyLlama-1.1B-Chat-v1.0',
    device=device
)

# Exibe informações sobre o pipeline de geração de texto
print(tiny_llama)
# Exibe detalhes sobre o modelo de linguagem carregado no pipeline
print(tiny_llama.model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): 

In [ ]:
# Define um prompt inicial para geração de texto
prompt = "Today is a beautiful day to"

# Gera texto a partir do prompt com um máximo de 25 novos tokens
output = tiny_llama(prompt, max_new_tokens=25)

# Exibe o texto gerado e o número de tokens limitados
print(output)
print(output[0]['generated_text'])

# vamos tacar aqui https://platform.openai.com/tokenizer e ver como o GPT faria a tokenizacao
# Ele encontrou o mesmo numero de tokens que limitamos a geracao

[{'generated_text': 'Today is a beautiful day to be alive.\n\nVerse 2:\nThe sun is shining bright, the birds are singing,\nThe'}]
Today is a beautiful day to be alive.

Verse 2:
The sun is shining bright, the birds are singing,
The


In [ ]:
# vendo token a token o processo de geracao

# tensor eh o array que redes neurais utilizam. pt --> pytorch. tf--> tensorflow.
# soh o tensor tem o metodo to(device), que serve para mover o array para o mesmo
# dispositivo que o nosso modelo esta

# Codifica o prompt em tokens e move para o dispositivo correto (GPU ou CPU)
prompt_input_ids = tiny_llama.tokenizer.encode(prompt, return_tensors='pt').to(device)
print(prompt_input_ids)

# Gera o texto com informações detalhadas, incluindo pontuações dos tokens gerados
output = tiny_llama.model.generate(
    prompt_input_ids, max_new_tokens=25,
    return_dict_in_generate=True,  # Retorna um dicionário com informações detalhadas
    output_scores=True  # Retorna as pontuações de cada token gerado
)

print(output.keys())

tensor([[    1, 20628,   338,   263,  9560,  2462,   304]])
odict_keys(['sequences', 'scores', 'past_key_values'])


In [ ]:
# Calcula as pontuações de transição (probabilidades dos tokens gerados)
transitions = tiny_llama.model.compute_transition_scores(
    output.sequences, output.scores, normalize_logits=True
)

print(transitions)

tensor([[-1.5247e+00, -5.8344e-01, -9.5884e-01, -1.5707e+00, -1.4555e+00,
         -1.5399e+00, -9.6954e-04, -2.2908e-02, -1.0781e+00, -6.2057e-02,
         -1.6606e-01, -2.1464e+00, -6.8957e-01, -9.2513e-01, -2.5262e-01,
         -4.2558e-03, -7.1952e-01, -2.9928e-01, -1.0641e+00, -1.2072e+00,
         -3.4627e-02, -1.2238e-01, -4.7702e-01, -1.8247e-01, -5.4553e-01]])

In [ ]:
# Calcula o tamanho do prompt em tokens e o exibe
tamanho_prompt = len(prompt_input_ids[0])
print(tamanho_prompt)

# Exibe a sequência do prompt nos tokens gerados
print(output.sequences[0][:tamanho_prompt])

# Obtém apenas os tokens gerados pelo modelo (excluindo o prompt original)
generated_tokens = output.sequences[0][tamanho_prompt:]
print(generated_tokens)

# Exibe uma tabela com o ID do token, pontuação, string do token e probabilidade
print('| token id | score  | token str | prob % |')
print('|----------+--------+-----------+--------|')
for (token, score) in zip(generated_tokens, transitions[0]):
    if tiny_llama.tokenizer.decode(token) == '\n':
        continue
    print(f"| {token:8d} | {score.to('cpu').numpy():.3f} | {tiny_llama.tokenizer.decode(token):9s} | {np.exp(score.to('cpu').numpy()):.4f} |")

7
tensor([    1, 20628,   338,   263,  9560,  2462,   304])
tensor([  367, 18758, 29889,    13,    13,  6565,   344, 29871, 29906, 29901,
           13,  1576,  6575,   338,   528,  2827, 11785, 29892,   278, 17952,
          526, 23623, 29892,    13,  1576])
| token id | score  | token str | prob % |
|----------+--------+-----------+--------|
|      367 | -1.525 | be        | 0.2177 |
|    18758 | -0.583 | alive     | 0.5580 |
|    29889 | -0.959 | .         | 0.3833 |
|     6565 | -1.540 | Ver       | 0.2144 |
|      344 | -0.001 | se        | 0.9990 |
|    29871 | -0.023 |           | 0.9774 |
|    29906 | -1.078 | 2         | 0.3402 |
|    29901 | -0.062 | :         | 0.9398 |
|     1576 | -2.146 | The       | 0.1169 |
|     6575 | -0.690 | sun       | 0.5018 |
|      338 | -0.925 | is        | 0.3965 |
|      528 | -0.253 | sh        | 0.7768 |
|     2827 | -0.004 | ining     | 0.9958 |
|    11785 | -0.720 | bright    | 0.4870 |
|    29892 | -0.299 | ,         | 0.7413 |
|      27

In [ ]:
# Mede o tempo de execução para gerar um texto com 100 novos tokens
start = time.time()
output = tiny_llama(prompt, max_new_tokens=100)
end = time.time()

# Exibe o modelo e o tempo de execução
print(tiny_llama.model)
print(f'Tempo de execucao: {end - start:.2f} segundos')
print(output)
print(output[0]['generated_text'])

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): 

In [ ]:
# Gera texto a partir de dois prompts consecutivos para verificar a "memória" do modelo
prompt1 = "What day is today?"
prompt2 = "What day is tomorrow?"

# Gera a primeira resposta
output = tiny_llama(prompt1, max_new_tokens=10)
print(output[0]['generated_text'])
print("-"*80)

# Gera a segunda resposta
output = tiny_llama(prompt2, max_new_tokens=10)
print(output[0]['generated_text'])

# O modelo provavelmente "alucinou" e deu uma resposta inesperada


What day is today?

Student: It's Monday.

--------------------------------------------------------------------------------
What day is tomorrow?

Tomorrow is Monday.

Inter


In [ ]:
# para se lembrar, precisamos alimentar ele com TUDO QUE FOI GERADO ANTERIORMENTE

prompt1 = "What day is today?"
prompt2 = "What day is tomorrow?"

# Para que o modelo lembre-se do contexto, precisamos alimentar tudo o que foi gerado anteriormente
output = tiny_llama(prompt1, max_new_tokens=10)
print(output[0]['generated_text'])

print("-"*80)

# Alimenta a resposta anterior e o segundo prompt para gerar texto coerente
output = tiny_llama(output[0]['generated_text'] + prompt2, max_new_tokens=30)
print(output[0]['generated_text'])

What day is today?

Student: It's Monday.

--------------------------------------------------------------------------------
What day is today?

Student: It's Monday.
. What day is tomorrow?

Student: It's Tuesday.
. What day is next week?

Student: It's Wednesday.


In [ ]:
# vamos carregar outro modelo (alucinou demais anteriormente)

# O código abaixo vai explodir a memória ram (pq ja temos o outro modelo carregado nela).
# deletando e limpando a memória

# O código abaixo vai liberar a memória ao remover o modelo TinyLlama da memória para carregar o modelo Falcon
# import gc  # Garbage Collector
# del tiny_llama
# gc.collect()
# torch.cuda.empty_cache()

# Carregamento do modelo Falcon (cuidado com o uso de memória)
# falcon = transformers.pipeline(
#     task='text-generation',
#     model='tiiuae/falcon-7b',
#     device=device
# )